In [1]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/tmp/,tmp/,0


In [2]:
dbutils.fs.ls("dbfs:/FileStore/aws")

Out[21]: [FileInfo(path='dbfs:/FileStore/aws/aws_cred.json', name='aws_cred.json', size=111)]

In [3]:
import json
file_location = '/dbfs/FileStore/aws/aws_cred.json'
with open(file_location, 'r') as json_data_file:
  config = json.load(json_data_file)

--------------------------------------------------------------------------- 
 FileNotFoundError Traceback (most recent call last)
 <command-4102582210486241> in <module> 
 1 import json
 2 file_location = '/dbfs/FileStore/aws/aws_cred.json' 
 ----> 3 with open ( file_location , 'r' ) as json_data_file : 
 4 config = json . load ( json_data_file ) 

 FileNotFoundError : [Errno 2] No such file or directory: '/dbfs/FileStore/aws/aws_cred.json'

In [4]:
S3_RESOURCE = 's3'
SCHEME = 's3a'

### BEGIN STRIP ###
BUCKET_NAME = 'lead-us-car'
PREFIX = ''
### END STRIP ###

INPUT_FILENAME = 'US_Accidents_June20.csv'

In [5]:
ACCESS_KEY="_"
SECRET_KEY="_"

### Add your credentials to Spark 
sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", ACCESS_KEY)
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", SECRET_KEY)

In [6]:
# This is just a utility function
def get_s3_path(key, bucket_name=BUCKET_NAME, scheme=SCHEME):
    return f"{scheme}://{bucket_name}/{key}"

In [7]:
filepath = get_s3_path(f'{PREFIX}/{INPUT_FILENAME}')
filepath

Out[7]: 's3a://lead-us-car//US_Accidents_June20.csv'

In [8]:
us_car_accidents = (spark.read.format('csv')
                    .option('header', 'true')
                    .option('inferSchema', 'true')
                    .load(filepath))

In [9]:
# schema
us_car_accidents.printSchema()

root
-- ID: string (nullable = true)
-- Source: string (nullable = true)
-- TMC: double (nullable = true)
-- Severity: integer (nullable = true)
-- Start_Time: string (nullable = true)
-- End_Time: string (nullable = true)
-- Start_Lat: double (nullable = true)
-- Start_Lng: double (nullable = true)
-- End_Lat: double (nullable = true)
-- End_Lng: double (nullable = true)
-- Distance(mi): double (nullable = true)
-- Description: string (nullable = true)
-- Number: double (nullable = true)
-- Street: string (nullable = true)
-- Side: string (nullable = true)
-- City: string (nullable = true)
-- County: string (nullable = true)
-- State: string (nullable = true)
-- Zipcode: string (nullable = true)
-- Country: string (nullable = true)
-- Timezone: string (nullable = true)
-- Airport_Code: string (nullable = true)
-- Weather_Timestamp: string (nullable = true)
-- Temperature(F): double (nullable = true)
-- Wind_Chill(F): double (nullable = true)
-- Humidity(%): double (nullable = true)
-- Pressure(in): double (nullable = true)
-- Visibility(mi): double (nullable = true)
-- Wind_Direction: string (nullable = true)
-- Wind_Speed(mph): double (nullable = true)
-- Precipitation(in): double (nullable = true)
-- Weather_Condition: string (nullable = true)
-- Amenity: boolean (nullable = true)
-- Bump: boolean (nullable = true)
-- Crossing: boolean (nullable = true)
-- Give_Way: boolean (nullable = true)
-- Junction: boolean (nullable = true)
-- No_Exit: boolean (nullable = true)
-- Railway: boolean (nullable = true)
-- Roundabout: boolean (nullable = true)
-- Station: boolean (nullable = true)
-- Stop: boolean (nullable = true)
-- Traffic_Calming: boolean (nullable = true)
-- Traffic_Signal: boolean (nullable = true)
-- Turning_Loop: boolean (nullable = true)
-- Sunrise_Sunset: string (nullable = true)
-- Civil_Twilight: string (nullable = true)
-- Nautical_Twilight: string (nullable = true)
-- Astronomical_Twilight: string (nullable = true)

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, FloatType

In [11]:
# select useful cols
us_car_accidents_final = us_car_accidents.select(
  F.col('State').cast(StringType()),
  F.col('Severity').cast(IntegerType()),
  F.month('Start_Time').alias('month'),
  F.year('Start_Time').alias('year')
)

In [12]:
display(us_car_accidents_final.head(5))

State,Severity,month,year
OH,3,2,2016
OH,2,2,2016
OH,2,2,2016
OH,3,2,2016
OH,2,2,2016


In [13]:
us_car_accidents_final.count()

Out[24]: 3513617

In [14]:
# check for null values
display(us_car_accidents_final.describe())

summary,State,Severity,month,year
count,3513617,3513617,3513617,3513617
mean,null,2.3399286262560772,6.525489829995699,2018.1404342021342
stddev,null,0.5521934519055779,3.4872696030617356,1.2398291446768033
min,AL,1,1,2016
max,WY,4,12,2020


In [15]:
### GroupBy 
us_car_accident_grouped = (us_car_accidents_final
                           .groupby(['State'])
                           .count()
                           .orderBy(F.desc('count')))

### Grouped count
display(us_car_accident_grouped)

State,count
CA,816825
TX,329284
FL,258002
SC,173277
NC,165958
NY,160817
PA,106787
IL,99692
VA,96075
MI,95983


In [16]:
BUCKET_NAME="lead-us-car"
DIRECTORY="final"

(us_car_accident_grouped
 .coalesce(1) # to force saving in one file
 .write
 .format("com.databricks.spark.csv")
 .option("header", "true")
 .save("s3a://{}:{}@{}/{}".format(ACCESS_KEY, SECRET_KEY, BUCKET_NAME, DIRECTORY)))